In [1]:

import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.append(str(parent_dir))

import numpy as np
from tqdm import tqdm
import torch

from datasets import load_dataset
from src.dataset import Dataset
from src.model_dump import Dump
from src.metrics import evaluate
from src.model_phase import PhraseModel
from src.model_query import QueryModel

raw_datasets = load_dataset("squad")

/home/ubuntu/roseltorg_devs/research/developers/Artem-folder/DensePhrasesOld/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_train = Dataset(model_checkpoint='SpanBERT/spanbert-base-cased')
dataset_train.load_ds(raw_datasets['train'], size=100)

100%|██████████| 100/100 [00:00<00:00, 1678.63it/s]


In [3]:
dataset_dev = Dataset(model_checkpoint='SpanBERT/spanbert-base-cased')
dataset_dev.load_ds(raw_datasets['validation'], size=10)

100%|██████████| 10/10 [00:00<00:00, 1807.58it/s]


In [4]:
# lens = []
# for i in range(len(dataset.contexts)):
#     context_ids = dataset.tokenizer(dataset.contexts[i])['input_ids'][1:-1]
#     answer_ids = dataset.tokenizer(dataset.answers[i][0])['input_ids'][1:-1]
#     # print(dataset.contexts[i])
#     # print(dataset.answers[i][0])
#     # print(context_ids)
#     # print(answer_ids)
#     # start = context_ids.index(answer_ids[0]) 
#     # end = context_ids.index(answer_ids[-1])
    
#     start = dataset.spans_input_ids[i]['start']
#     end = dataset.spans_input_ids[i]['end']
#     print(len(context_ids), start, end)
#     print(dataset.tokenizer.decode(context_ids[start:end]), end='\n\n')

In [5]:
phrasemodel = PhraseModel(hidden_dim=768, ds=dataset_train, model_checkpoint='SpanBERT/spanbert-base-cased', device='cpu')


Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
   
L = len(dataset_train.contexts)
indices = list(range(L))
np.random.shuffle(indices)
batch_size = 5
inputs_batched_train = [indices[i*batch_size:(i+1)*batch_size]
                          for i in range(L //batch_size)]

   
L = len(dataset_dev.contexts)
indices = list(range(L))
np.random.shuffle(indices)
inputs_batched_dev = [indices[i*batch_size:(i+1)*batch_size]
                          for i in range(L //batch_size)]

In [7]:
optimizer = torch.optim.Adam(phrasemodel.parameters(), lr=0.1)

num_epochs = 1
for epoch in range(num_epochs):
    # Forward pass
    losses = []
    for inputs in tqdm(inputs_batched_train):
        optimizer.zero_grad()
        loss = phrasemodel(inputs)

        # # Backward and optimize
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
    print(np.mean(losses))

  0%|          | 0/20 [00:00<?, ?it/s]/home/ubuntu/roseltorg_devs/research/developers/Artem-folder/DensePhrasesOld/src/model_phase.py:51: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3637.)
  z_start = torch.matmul(last_hidden_state[n], q_start[n].T).reshape(context_num_tokens)
/home/ubuntu/roseltorg_devs/research/developers/Artem-folder/DensePhrasesOld/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
 10%|█         | 2/20 [00:12<01:52,  6.28s/it]


KeyboardInterrupt: 

In [8]:
dump = Dump(dataset_train, phrasemodel.model, 
              phrasemodel.model_start, phrasemodel.model_end,
              phrasemodel.tokenizer, hidden_dim=768, 
              device='cpu')

dump.create_dump()

Creating Phrase dump: 100%|██████████| 21/21 [00:04<00:00,  4.22it/s]


In [9]:
ids = [0]
hidden_dim = 768
N = len(ids)
k = 100
questions = dataset_train.get_questions(ids)
contexts = dataset_train.get_contexts(ids)

input_ids = dataset_train.tokenizer(questions, truncation=True, max_length=512, padding=True, return_tensors="pt")
last_hidden_state_start = dump.model_start(**input_ids).last_hidden_state.detach().cpu().numpy()[:, 0, :].reshape((N, hidden_dim))
last_hidden_state_end = dump.model_end(**input_ids).last_hidden_state.detach().cpu().numpy()[:, 0, :].reshape((N, hidden_dim))
S_start, I_start = dump.index.search(last_hidden_state_start, k)

In [10]:
I_start

array([[2212,  405, 2731, 2448, 2398, 1636, 1596,  916,  644, 4081, 3838,
        3347, 3236, 2785, 2286, 2248, 2239, 1550, 1251, 1202, 1138,  974,
         850,  774,  598,  444, 4252, 4239, 4085, 4083, 4072, 4062, 3989,
        3856, 3844, 3755, 3566, 3207, 3206, 3200, 3098, 3090, 2986, 2798,
        2793, 2754, 2666, 2416, 2332, 2290, 2251, 2199, 2171, 1614, 1565,
        1527, 1510, 1474, 1430, 1328, 1326, 1302, 1242, 1195, 1169, 1156,
        1131, 1081, 1044, 1036,  996,  894,  891,  869,  852,  791,  710,
         655,  653,  651,  646,  635,  628,  617,  616,  576,  518,  478,
         367,  329,  119,  788,  707,  691,  636,  623,  605,  566,  556,
         521]])

In [13]:
dump.predict([3])

KeyError: -1

In [8]:
query_side_model = QueryModel(phrasemodel.tokenizer,
              hidden_dim=768, dump=dump, ds=dataset_train,
              device='cpu')

In [9]:
query_side_optimizer = torch.optim.Adam(query_side_model.parameters(), lr=0.01)

num_epochs = 5
for epoch in range(num_epochs):
    # Forward pass
    losses = []
    for inputs in tqdm(inputs_batched_train):
        query_side_optimizer.zero_grad()
        loss = query_side_model(inputs)

        # # Backward and optimize
        loss.backward()
        torch.nn.utils.clip_grad_norm_(query_side_model.parameters(), 1.0)
        query_side_optimizer.step()
        losses.append(loss.item())
        
    print(f"loss: {np.mean(losses)}")
    
    query_side_model.eval()
    em_strings = []
    em_starts = []
    em_ends = []
    delta_start_normalizeds = []
    delta_end_normalizeds = []
    with torch.no_grad():
        for inputs in tqdm(inputs_batched_dev):
            prediction_answers, prediction_start_indices, prediction_end_indices, _ = \
                query_side_model.dump.predict(inputs)
            
            em_string, em_start, em_end, delta_start_normalized, delta_end_normalized = \
                evaluate(dataset_dev.get_answers(inputs), dataset_dev.get_spans_input_ids(inputs, pos='start'), 
                         dataset_dev.get_spans_input_ids(inputs, pos='end'),dataset_dev.get_contexts(inputs), 
                    prediction_answers, prediction_start_indices, prediction_end_indices)
            em_strings.append(em_string)
            em_starts.append(em_start)
            em_ends.append(em_end)
            delta_start_normalizeds.append(delta_start_normalized)
            delta_end_normalizeds.append(delta_end_normalized)
    
    print(f"em_strings {np.mean(em_strings)}")
    print(f"em_starts {np.mean(em_starts)}")
    print(f"em_ends {np.mean(em_ends)}")
    print(f"delta_start_normalizeds {np.mean(delta_start_normalizeds)}")
    print(f"delta_end_normalizeds {np.mean(delta_end_normalizeds)}")
    

 15%|█▌        | 3/20 [02:24<13:38, 48.14s/it]


KeyboardInterrupt: 

In [ ]:
for i in range(100):
    answer, start_index, end_index, score = dump.predict(i, k=100, verbose=True)
    print(dataset.spans_input_ids[i]['start'], start_index)
    print(dataset.spans_input_ids[i]['end'], end_index)
    print(answer)
    print()

Q: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
C: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
128 40
136 40


Q: What is in front of the Notre Dame Main Building?
C: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of th

KeyboardInterrupt: 